In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-connect-2025/Test_100_Puzzles.parquet
/kaggle/input/ai-connect-2025/README.md
/kaggle/input/ai-connect-2025/Gridmode-00000-of-00001.parquet
/kaggle/input/ai-connect-2025/Test_100_Puzzles.csv
/kaggle/input/ai-connect-2025/mc-00000-of-00001.parquet
/kaggle/input/ai-connect-2025/gitattributes.txt


In [2]:
import pandas as pd

In [3]:
zebraLogicBench = pd.read_parquet('/kaggle/input/ai-connect-2025/Gridmode-00000-of-00001.parquet')
zebraLogicBench

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-03T21:21:29.209499
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-03T21:21:29.207505
2,lgp-test-6x4-15,6*4,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Children', 'Musi...",2024-07-03T21:21:29.210360
3,lgp-test-6x5-2,6*5,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Mother', 'Childr...",2024-07-03T21:21:29.210554
4,lgp-test-2x2-33,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Pet'], 'rows': [...",2024-07-03T21:21:29.204640
...,...,...,...,...,...
995,lgp-test-3x4-1,3*4,"There are 3 houses, numbered 1 to 3 from left ...","{'header': ['House', 'Name', 'Animal', 'Cigar'...",2024-07-03T21:21:29.206002
996,lgp-test-5x5-39,5*5,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Birthday', 'Moth...",2024-07-03T21:21:29.209334
997,lgp-test-2x2-25,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Vacation'], 'row...",2024-07-03T21:21:29.204603
998,lgp-test-2x4-34,2*4,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'PhoneModel', 'Oc...",2024-07-03T21:21:29.205053
